Install

In [1]:
!pip install boto3 s3fs
!pip install gradio


INFO: pip is looking at multiple versions of aiobotocore to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of aiobotocore to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.8/104.8 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 kB 2.1 MB/s eta 0:00:00
  Preparing

Imports

In [2]:

import boto3
import json
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_squared_error
import gradio as gr

print("All libraries loaded")

All libraries loaded


In [3]:
# Configurar as credenciais de acesso
aws_access_key = ""
aws_secret_key = ""
aws_session_token = ""

# Conectar com o S3
s3_client = boto3.client(
    's3',
    aws_access_key_id=aws_access_key,
    aws_secret_access_key=aws_secret_key,
    aws_session_token=aws_session_token
)

# Baixar o arquivo diretamente
bucket_name = "btc-data-bronze"
json_key = "dados/dados.json"

response = s3_client.get_object(Bucket=bucket_name, Key=json_key)
json_content = response['Body'].read().decode('utf-8')

ClientError: An error occurred (ExpiredToken) when calling the GetObject operation: The provided token has expired.

In [ ]:

data_dict = json.loads(json_content)

# Extrai a seção 'Time Series (Digital Currency Daily)'
time_series = data_dict['Time Series (Digital Currency Daily)']

# Cria uma lista de tuplas com os dados
rows = []
for date, values in time_series.items():
    rows.append((date, float(values['1. open']), float(values['2. high']), float(values['3. low']), float(values['4. close']), float(values['5. volume'])))

# Define o schema do DataFrame
columns = ["date", "open", "high", "low", "close", "volume"]

df = pd.DataFrame(rows, columns=columns)

In [ ]:
# Converter a coluna 'date' para datetime
df['date'] = pd.to_datetime(df['date'])

# Reverter a ordem dos dados para que a data mais recente esteja no início
df = df[::-1].reset_index(drop=True)

# Criar médias móveis
df['SMA_7'] = df['close'].rolling(window=7).mean()
df['SMA_30'] = df['close'].rolling(window=30).mean()

# Calcular a volatilidade (desvio padrão de 7 dias)
df['volatility'] = df['close'].rolling(window=7).std()

# Diferença de fechamento (retorno diário)
df['daily_diff'] = df['close'].diff()

In [ ]:
# Remover NaN (devido aos cálculos das médias móveis)
data = df.dropna()

# Criar as variáveis de features e target
features = ['open', 'high', 'low', 'volume', 'SMA_7', 'SMA_30', 'volatility', 'daily_diff']
X = data[features]
y = data['close']

In [ ]:
# Normalizar os dados
scaler = MinMaxScaler(feature_range=(0, 1))
X_scaled = scaler.fit_transform(X)
y_scaled = scaler.fit_transform(y.values.reshape(-1, 1))

# Dividir os dados em treino e teste (70% treino, 30% teste)
train_size = int(len(df) * 0.70)
X_train, X_test = X_scaled[:train_size], X_scaled[train_size:]
y_train, y_test = y_scaled[:train_size], y_scaled[train_size:]

In [ ]:
# Gerar janelas temporais de 30 dias para treinar o modelo
n_input = 60  # Tamanho da janela temporal
train_generator = TimeseriesGenerator(X_train, y_train, length=n_input, batch_size=15)
test_generator = TimeseriesGenerator(X_test, y_test, length=n_input, batch_size=15)

In [ ]:
# Ajuste de Hiperparâmetros
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(n_input, X_train.shape[1]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')
history = model.fit(train_generator, epochs=150, validation_data=test_generator)


In [ ]:
# Fazer previsões nos dados de teste
y_pred = model.predict(test_generator)

# Reverter a normalização dos dados
y_pred_rescaled = scaler.inverse_transform(y_pred)
y_test_rescaled = scaler.inverse_transform(y_test[n_input:])

# Avaliar o modelo usando o erro quadrático médio
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test_rescaled, y_pred_rescaled)
print(f'Mean Squared Error: {mse}')

# Previsão para 7 Dias
forecast_days = 7
input_sequence = X_scaled[-n_input:].reshape(1, n_input, X_scaled.shape[1])

predictions = []

for _ in range(forecast_days):
    pred = model.predict(input_sequence)
    predictions.append(pred[0][0])
    temp_array = np.zeros((1, 1, X_scaled.shape[1]))
    temp_array[0, 0, -1] = pred[0][0]
    input_sequence = np.concatenate([input_sequence[:, 1:, :], temp_array], axis=1)

predictions_rescaled = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))
last_date = df['date'].iloc[-1]
forecast_dates = pd.date_range(last_date, periods=forecast_days + 1, freq='D')[1:]


# Exibir as previsões dos próximos 7 dias
print("Previsões para os próximos 7 dias:")
print(predictions_rescaled)  # Print the rescaled predictions

# Plotando as previsões
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(forecast_dates, predictions_rescaled, label='Previsões', color='blue')
plt.title('Previsões para os Próximos 7 Dias')
plt.xlabel('Data')
plt.ylabel('Valor Previstos')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Visualização
# --- Creating train_data and test_data DataFrames ---
# Assuming 'train_size' is already defined from previous code
train_data = df[:train_size]  # Create train_data DataFrame
test_data = df[train_size:]   # Create test_data DataFrame


plt.figure(figsize=(10,6))
plt.plot(df['date'], df['close'], label='Histórico Completo', color='blue')
plt.plot(train_data['date'], train_data['close'], label='Dados de Treinamento', color='green')
plt.plot(test_data['date'], test_data['close'], label='Dados de Teste', color='orange')

# Plotagem das Previsões em Cinza
start_index = len(train_data) + n_input
end_index = start_index + len(y_pred_rescaled)
prediction_dates = df['date'][start_index:end_index]

plt.plot(prediction_dates, y_pred_rescaled, label='Previsões em Teste', color='gray')

# Conectar Linha com Último Ponto de Teste
plt.plot([test_data['date'].iloc[-1], forecast_dates[0]],
         [test_data['close'].iloc[-1], predictions_rescaled[0][0]],
         color='red')

# Previsão para os Próximos 7 Dias
plt.plot(forecast_dates, predictions_rescaled, label='Previsões para 7 Dias', color='red', linestyle='dashed')


plt.xlabel('Data')
plt.ylabel('Preço de Fechamento (USD)')
plt.title('Histórico de Preço de Criptomoeda com Previsões Futuras')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
def plot_historical():
    plt.figure(figsize=(10,6))
    plt.plot(df['date'], df['close'], label='Histórico Completo', color='blue')
    plt.plot(train_data['date'], train_data['close'], label='Dados de Treinamento', color='green')
    plt.plot(test_data['date'], test_data['close'], label='Dados de Teste', color='orange')

    # Plotagem das Previsões em Cinza
    start_index = len(train_data) + n_input
    end_index = start_index + len(y_pred_rescaled)
    prediction_dates = df['date'][start_index:end_index]

    plt.plot(prediction_dates, y_pred_rescaled, label='Previsões em Teste', color='gray')

    # Conectar Linha com Último Ponto de Teste
    #plt.plot([test_data['date'].iloc[-1], forecast_dates[0]],
    #         [test_data['close'].iloc[-1], predictions_rescaled[0]],  # Ajustado aqui
    #         color='red')

    # Previsão para os Próximos 7 Dias
    #plt.plot(forecast_dates, predictions_rescaled, label='Previsões para 7 Dias', color='red', linestyle='dashed')

    plt.xlabel('Data')
    plt.ylabel('Preço de Fechamento (EUR)')
    plt.title('Histórico de Preço de Criptomoeda')
    plt.legend()
    plt.xticks()
    plt.tight_layout()
    return plt

def plot_forecast():
    plt.figure(figsize=(10, 6))
    plt.plot(forecast_dates, predictions_rescaled, label='Previsões', color='blue')
    plt.title('Previsões para os Próximos 7 Dias')
    plt.xlabel('Data')
    plt.ylabel('Valor Previsto')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    return plt

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("# Histórico de valores de Bitcoin e previsão de tendência para próxima semana")
    gr.Plot(plot_historical, label="Histórico")
    gr.Plot(plot_forecast, label="Previsões")

demo.launch(share=True)